# BackTestのチュートリアル

<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/tutorial/breakout_agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

　
## 1. 実装するBotのロジック

Ukiさんの「オープニングレンジ・ブレイクアウト」をお借りして、バックテストのtutorialを作成しました（公開くさだって感謝）。

Binanceのspot/BTCBUSD用に書いていますが、PublicAPIのみ利用のためアカウントは不要で試すことができます。



<blockquote class="twitter-tweet"><p lang="ja" dir="ltr">お待ちかねのロジックを公開。<br>(元祖)ドテン君はブレイクアウトだが通常のHLチャネルブレイクではない。その手法は「オープニングレンジ・ブレイクアウト」と呼ばれるものである。文章での説明は面倒なのでhohetoとの会議資料をそのまま添付する。ストラテジーに関する質問には応対できません。 <a href="https://t.co/LB6mdxVZZo">pic.twitter.com/LB6mdxVZZo</a></p>&mdash; UKI (@blog_uki) <a href="https://twitter.com/blog_uki/status/981768546429448192?ref_src=twsrc%5Etfw">April 5, 2018</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

![](https://pbs.twimg.com/media/DZ_xJQFUMAEOrpS?format=jpg&name=900x900)

## 2. BreakOutAgentの実装

日本語で読み解くと以下のようになります。

「シグナル点灯した次の足の始値」ではなく、シグナル点灯直後の最良値でエントリーすることに変更した以外はほぼ忠実に書き下しています。

1. `Agent.on_clock`を10分毎に呼び出す。
2. 

3. `Agent.on_clock`内で以下の処理を行う。

    1. 前処理
       
       前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）
    2. Long/Short判定
      * 現在時刻から2時間足を6本取得する。`session.ohlcv`（6本目の最後の足は未確定足。10分毎に呼ばれるたびにupdateされる）
      (ohlcv[0]-ohlcv[5]へ値を格納)
      * 0-4本目の足のレンジ幅（高値ー安値）の平均値を計算`(ohlcv['high]-ohlcv['low']).mean() * K`し、Kをかけたものを'range_width'へ保存する。
      * 最新足（未確定）の始値〜高値を計算する('diff_high')
      * 最新足（未確定）の始値〜安値を計算する('diff_low')
      * 'range_width', 'diff_high', 'diff_low'の結果からLong/Short判定
    3. オーダー執行
       1. Long判定のとき
          * すでにLongポジションがある場合は何もしない。
          * Shortポジションがあった場合はドテン（最良値、倍サイズ）
          * 上記に当てはまらない場合は最良値、通常サイズでLong(Buy)
       2. Short判定のとき
           * すでにShortポジションがある場合は何もしない
           * Shortポジションがあった場合はドテン（最良値、倍サイズ）
           * 上記に当てはまらない場合は最良値、通常サイズでShort(Sell)

## 2. 依存ライブラリのインストール

In [ ]:
# rbot拡張ライブラリのインストール（２回目は不要です）
! pip install --upgrade pip
! pip install --upgrade rbot

In [ ]:
# 関連ライブラリのインストール
! pip install pyarrow
! pip install polars
! pip install plotly
! pip install nbformat
! pip install numpy
! pip install pandas
! pip install json2html

In [ ]:
# rbotは今回提供するbacktestライブラリ
import rbot
from rbot import BinanceMarket
from rbot import BinanceConfig
from rbot import Runner

from rbot import time_string
from rbot import DAYS_BEFORE

## 3. break out Agent(bot)の実装

#### 時間の扱い

時刻について、以下の基準で設計しています。
* 時刻（ある一点を指す）: UTCでエポックタイムからのマイクロ秒(μs)。時刻０はfromの時は過去全部、toのときは将来全部（≒現在時刻）
* 期間：ほとんどで秒(s)。ログのダウンロード`Market.download`だけ日(day)
  * 例：ohlcvの足の幅などは秒を利用。

#### Botの実装すべき必須メソッド

３つのメソッドを実装するとフレームワークが適宜呼び出してくれます。
* `on_clock(self, session, current_time_us)` 定期的に呼び出される関数です。`time_us`は現在時刻です。`session`クラスを使ってローソク足`ohlcv`をとったりオーダーの発行ができます。
* `on_tick(self, session, side, price, size)` 全ログイベントを受け取ります。
* `on_update(self, session, updated_order)`オーダーが約定したり失効した場合に呼ばれます。'updated_order'の中に更新後のオーダ情報が含まれています。



In [ ]:
class BreakOutAgent:
    """
        Agentのクラス名は任意。
        on_tick, on_clock, on_updateを実装するとフレームワークから呼び出される。
    """

    def __init__(self, param_K=1.6):
            self.param_K = param_K  # パラメターKを設定する。
            self.offset = 10.0       # 指値のオフセット

    def on_init(self, session):
        print("execute mode = ", session.execute_mode)
        session.clock_interval_sec = 60*60*2  # 2時間ごとにon_clockを呼び出すように設定する。

    def on_clock(self, session, time_us):
        """ Botのメインロジック。設定した秒数毎に呼ばれる """
        # 古いオーダーをキャンセルする。
        
        # 前処理/ 前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）        
        if session.buy_orders or session.sell_orders:
            session.expire_order(60*60*2)  # 2時間以上経過したオーダーをキャンセルする。
        
        ############   メインロジック  ###################### 
        ohlcv_df = session.ohlcv(60*60*2, 6)  # 2時間足(60*60*2sec)を６本取得。 最新は６番目。ただし未確定足
        if len(ohlcv_df) < 6:           # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv5 = ohlcv_df[:-2]       # 過去５本足（確定）
        range_width = (ohlcv5['high'] - ohlcv5['low']).mean() * self.param_K  # 価格変動レンジの平均を計算 * K

        # Long/Short判定
        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本(未確定)        
        diff_low   =   (ohlcv_latest['open'][0] - ohlcv_latest['low'][0])
        detect_short  = range_width < diff_low

        diff_high  = - (ohlcv_latest['open'][0] - ohlcv_latest['high'][0])  
        detect_long = range_width  < diff_high
        
        ##########  メインロジック中に利用したindicatorのロギング（あとでグラフ化するため保存）    ##############
        session.log_indicator('diff_low', diff_low)
        session.log_indicator('diff_high', diff_high)
        session.log_indicator('range_width', range_width)

        ##########　執行戦略（順方向のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）#########
        ORDER_SIZE = 0.01     # 標準オーダーサイズ(ドテンの場合はx2)

        session.log_indicator('position', session.position)

        if detect_long and (session.position <= 0): 
            if session.sell_orders:
                for order in session.sell_orders:
                    session.cancel_order(order.order_id)
            
            bit_edge, ask_edge = session.last_price
            session.limit_order('Buy', bit_edge - self.offset, ORDER_SIZE + (-session.position))

        if detect_short and (0 <= session.position): # short判定のとき
            if session.buy_orders:
                for order in session.buy_orders:
                    session.cancel_order(order.order_id)

            bit_edge, ask_edge = session.last_price            
            session.limit_order('Sell', ask_edge + self.offset, ORDER_SIZE + session.position)

    # 全Tick受け取りたい時は on_tick を実装する。
    #def on_tick(self, time, session, side, price, size):
    #    pass

    #約定イベントを受け取りたい時は on_updateを実装する。
    def on_update(self, session, order_result):
        print(order_result)

    def on_account_update(self, session, account):
        print(account)


## 4. Binanceオブジェクトの作成データのロード

### Binanceオブジェクトを生成。

接続先、取り扱い通過ペアーなどの情報を`BinanceConfig`に設定します。あらかじめプリセットされた値として、BTCUSDペアー本番用`BinanceConfig.BTCUSDT`, BTCUSDTペアーテストネット用`BinanceConfig.TEST_BTCUSDT`が提供されていますので必要に応じて選択してください。まずはTESTネットでテストします。

BinanceMarketオブジェクトをJupyterで表示するとデータベースに保存されている取引履歴の期間などが表示されます。

In [ ]:
from rbot import BinanceConfig
from rbot import BinanceMarket

binance = BinanceMarket(BinanceConfig.BTCUSDT)
binance

### データのダウンロード

最初にダウンロードメソッドをつかってデータをダウンロードします。

#### ダウンロードメソッド
```
BinanceMarket.download(*, ndays, force=False, verbose=True)
```

##### パラメータ
* `ndays` 何日前のデータからダウンロードするかを指定します。
* `forde` オプション：TureにするとローカルDBにデータがあっても再ダウンロードします。
* `verbose` オプション:ダウンロード状況を印刷します（ディフォルトは表示）

##### 戻り値
* ダウンロードされたレコード数


In [ ]:
#from rbot import init_debug_log;
#init_debug_log()

# 過去２日分のデータをダウンロード
binance.download(
    ndays=10,       #ダウンロード日数
    verbose=True    #ダウンロード状況を表示
)

In [ ]:
# かなり巨大なデータベースができあがります。１日分で数百MB。
# DBを削除する場合は以下のコマンドを実行してください
# なお最後に表示されたファイルを別途OSから削除してください。
#binance.drop_table()

# DBのある場所を表示します。かなり大きなファイルになりますので不要になった場合手動で消してください。
#binance.file_name

## 5. バックテスト実行

In [ ]:

from rbot import NOW, DAYS

agent = BreakOutAgent()  # エージェントを作成する。パラメターKはデフォルトの1.6を利用する。

runner = Runner()

session = runner.back_test(
                market=binance,     # マーケットを指定する。
                agent=agent,    # エージェントを指定する。
                start_time=NOW()-DAYS(10),   # 開始時間を指定する。0は最初から。10日前から実行を指定。
                end_time=0,     # 終了時間を指定する。0は最後まで。
                verbose=True    # ログを表示するかどうか
)


### 実行結果の分析

`session.log`にLoggerオブジェクトとして結果が保存されています。
詳しくは[Loggerのマニュアル](../manual.ipynb)を参照ください。

In [ ]:
# loggerオブジェクトの取得
log = session.log

In [ ]:
import polars as pl
pl.Config(fmt_str_lengths=50)

# オーダログの表示
orders = log.orders
orders

## 6.結果分析


In [ ]:
# 損益結果
orders['sum_profit'][-1]

In [ ]:
# 利益ピーク
orders['sum_profit'].max()

In [ ]:
# 利益最小
orders['sum_profit'].min()

In [ ]:
# １回の取引の最大利益
orders['total_profit'].max()

In [ ]:
# １回の取引の最大損失
orders['total_profit'].min()

In [ ]:
# オーダー状況分析
orders.group_by(['order_side', 'status']).count()

In [ ]:
# オーダーごとに集約
group_by_order = orders.group_by(['order_id']).agg(
    pl.col('symbol').first(), 
    pl.col('order_side').first(), 
    pl.col('status').last(), 
    pl.col('order_price').first(), 
    pl.col('order_size').first(), 
    pl.col('execute_size').sum(),
    pl.col('update_time').last(),
    pl.col('total_profit').sum()
).sort('update_time')


group_by_order

In [ ]:
lost = len(group_by_order.filter(pl.col('total_profit') < 0))
win = len(group_by_order.filter(pl.col('total_profit') > 0))

print(f'勝ち:{win} 負け:{lost} 勝率:{win/(win+lost)}')

## 6. グラフで確認

### まずは該当期間のOHLCV（ローソク足）チャートを書く

#### OHLCVデータの作成


In [ ]:
ohlcv = binance.ohlcv (
    runner.start_timestamp, # runnerの開始時間
    runner.last_timestamp,  # runnerの終了時間
    60*60*2                 # 2時間足
)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("indicator", "position", "profit", "candlestick"))

# row 1 (indicator)
diff_low = log['diff_low']
fig.add_trace(go.Scatter(x=diff_low['timestamp'], y=diff_low['diff_low'], name="diff_low"), row=1, col=1)

diff_high = log['diff_high']
fig.add_trace(go.Scatter(x=diff_low['timestamp'], y=diff_low['diff_low'], name="diff_high"), row=1, col=1)

range_width = log['range_width']
fig.add_trace(go.Scatter(x=range_width['timestamp'], y=range_width['range_width'], name="range_width"), row=1, col=1)

orders = log.orders

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position"), row=2, col=1)


# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit"), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

fig.update_layout(height=800, title_text="Backtest Result")


## 7. まとめ

自分で書くのは百行ぐらいで簡単にバックテストができることが確認できました。

使ってみてご意見お聞かせください。
Twitter(@yasstake)でもgithubでissueあげてもOK.

人気になったらDiscord立ち上げてみたいなー。どおでしょ？


## 8. Next Step

いろいろバックテストしてみよう

#### BreakOutAgentのパラメータ変更
以下のパラメータを変えたらどうなるか確認してみよう
1. パラメータK (現在は1.6きめうち)
2. 指値（現在はbestプライス。
   * buyのときに低い価格で指すと利益は上がるがExpire率が上がる。
   * 高い価格で指すとExpire率が下がるが利益が下がる。（この場合、本来はTakerFeeが取られるが現在はMakerFeeで計算）
3. ロット（現在は0.01 BTC)の変更
4. 2時間足から他の足を試してみる。
5. Clockのタイミングを増やす・へらす（現在は10分）
6. バックテスト期間の延長（現在はBACKTEST_PERIODは5日で設定）

さらに独自のIndicatorをつくり、独自ロジックをつくってバックテストしてみよう（OHLCVをベースに判断するロジックならば問題なく作れるはず）